In [1]:
import ambulance_game as abg
import numpy as np


# Others


In [2]:
lambda_2 = 0.2
lambda_1 = 0.3
mu = 0.2
num_of_servers = 3
threshold = 4
system_capacity = 5
buffer_capacity = 1

output = "others"


$$
    \sum_{(u,v) \, \in S_A} \frac{1}{C \mu} \times max(v-C+1,0) \times \pi(u,v)
$$

In [7]:
def get_closed_form_waiting_time(
    lambda_2,
    lambda_1,
    mu,
    num_of_servers,
    threshold,
    system_capacity,
    buffer_capacity,
    output="both",
):

    all_states = abg.markov.build_states(
        threshold=threshold,
        system_capacity=system_capacity,
        buffer_capacity=buffer_capacity,
    )
    transition_matrix = abg.markov.get_transition_matrix(
        lambda_2=lambda_2,
        lambda_1=lambda_1,
        mu=mu,
        num_of_servers=num_of_servers,
        threshold=threshold,
        system_capacity=system_capacity,
        buffer_capacity=buffer_capacity,
    )
    pi = abg.markov.get_steady_state_algebraically(Q=transition_matrix)
    pi_array = abg.markov.get_markov_state_probabilities(pi, all_states)

    sojourn_time = 1 / (num_of_servers * mu)
    sojourn_array = (
        np.array(
            [
                [
                    states_to_wait
                    for states_to_wait in range(1, system_capacity - num_of_servers + 1)
                ]
            ]
        )
        * sojourn_time
    )  # for states_column in range(buffer_capacity + 1)]) * sojourn_time
    mean_waiting_time = np.nansum(
        pi_array[:, num_of_servers:-1] * sojourn_array
    ) / np.nansum(pi_array[:, :-1])
    return mean_waiting_time


In [8]:
get_closed_form_waiting_time(
    lambda_2=lambda_2,
    lambda_1=lambda_1,
    mu=mu,
    num_of_servers=num_of_servers,
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
)


1.2013571931626725

In [9]:
abg.markov.get_mean_waiting_time_markov(
    lambda_2=lambda_2,
    lambda_1=lambda_1,
    mu=mu,
    num_of_servers=num_of_servers,
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
    output=output,
)


1.2013571931626725

### Step-by-step


In [10]:
all_states = abg.markov.build_states(
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
)
transition_matrix = abg.markov.get_transition_matrix(
    lambda_2=lambda_2,
    lambda_1=lambda_1,
    mu=mu,
    num_of_servers=num_of_servers,
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
)
pi = abg.markov.get_steady_state_algebraically(Q=transition_matrix)
pi_array = abg.markov.get_markov_state_probabilities(pi, all_states)


In [11]:
# Sojourn time
sojourn_time = 1 / (num_of_servers * mu)
sojourn_time


1.6666666666666665

In [12]:
# Sojourn array
sojourn_array = np.array(
    [
        states_to_wait
        for states_to_wait in range(1, system_capacity - num_of_servers + 1)
    ]
)  # for states_column in range(buffer_capacity + 1)])
sojourn_array = sojourn_array * sojourn_time
sojourn_array


array([1.66666667, 3.33333333])

In [13]:
# Waiting states probability
pi_array[:, num_of_servers:-1]


array([[0.18632672, 0.15527227],
       [       nan, 0.07116646]])

In [199]:
# Waiting states probability x Sojourn array
pi_array[:, num_of_servers:-1] * sojourn_array


array([[0.31054454, 0.51757423],
       [       nan, 0.23722152]])

In [200]:
# Sum of the above matrix
np.nansum(pi_array[:, num_of_servers:-1] * sojourn_array)


1.0653402964320158

In [201]:
# Probability of accepting state
np.nansum(pi_array[:, :-1])


0.8867806365127919

In [202]:
# Mean waiting time = (Sum of the sojourn array multiplied by the probabilities of being at that state) / (Sum of probabilities of being at an accepting state)
np.nansum(pi_array[:, num_of_servers:-1] * sojourn_array) / np.nansum(pi_array[:, :-1])


1.2013571931626725

# Ambulance


In [186]:
lambda_2 = 0.2
lambda_1 = 0.3
mu = 0.15
num_of_servers = 4
threshold = 5
system_capacity = 7
buffer_capacity = 7

output = "ambulance"


$$
    \sum_{(u,v) \, \in S_A} \frac{1}{C \mu} \times max(min(v,T)-C+1,0) \times \pi(u,v)
$$

In [187]:
def get_closed_form_waiting_time(
    lambda_2,
    lambda_1,
    mu,
    num_of_servers,
    threshold,
    system_capacity,
    buffer_capacity,
    output="both",
):

    all_states = abg.markov.build_states(
        threshold=threshold,
        system_capacity=system_capacity,
        buffer_capacity=buffer_capacity,
    )
    transition_matrix = abg.markov.get_transition_matrix(
        lambda_2=lambda_2,
        lambda_1=lambda_1,
        mu=mu,
        num_of_servers=num_of_servers,
        threshold=threshold,
        system_capacity=system_capacity,
        buffer_capacity=buffer_capacity,
    )
    pi = abg.markov.get_steady_state_algebraically(Q=transition_matrix)
    pi_array = abg.markov.get_markov_state_probabilities(pi, all_states)

    sojourn_time = 1 / (num_of_servers * mu)
    sojourn_array = (
        np.array(
            [
                states_to_wait
                for states_to_wait in range(1, threshold - num_of_servers + 1)
            ]
            + [
                max(0, threshold - num_of_servers)
                for states_to_wait in range(threshold, system_capacity + 1)
            ]
        )
        * sojourn_time
    )

    if threshold > system_capacity:
        mean_waiting_time = np.nansum(
            pi_array[:, min(num_of_servers, threshold) : -1] * sojourn_array[:-1]
        ) / np.nansum(pi_array[:, :-1])
        return mean_waiting_time

    mean_waiting_time = np.nansum(
        pi_array[:-1, min(num_of_servers, threshold) :] * sojourn_array
    ) / np.nansum(pi_array[:-1, :])
    return mean_waiting_time


In [188]:
get_closed_form_waiting_time(
    lambda_2=lambda_2,
    lambda_1=lambda_1,
    mu=mu,
    num_of_servers=num_of_servers,
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
)


0.962897472757968

In [189]:
abg.markov.get_mean_waiting_time_markov(
    lambda_2=lambda_2,
    lambda_1=lambda_1,
    mu=mu,
    num_of_servers=num_of_servers,
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
    output=output,
)


0.9628974727579684

### Step-by-step


In [190]:
all_states = abg.markov.build_states(
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
)
transition_matrix = abg.markov.get_transition_matrix(
    lambda_2=lambda_2,
    lambda_1=lambda_1,
    mu=mu,
    num_of_servers=num_of_servers,
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
)
pi = abg.markov.get_steady_state_algebraically(Q=transition_matrix)
pi_array = abg.markov.get_markov_state_probabilities(pi, all_states)


In [191]:
# Sojourn time
sojourn_time = 1 / (num_of_servers * mu)
sojourn_time


1.6666666666666667

In [192]:
# Sojourn array
sojourn_array = np.array(
    [states_to_wait for states_to_wait in range(1, threshold - num_of_servers + 1)]
    + [
        max(0, threshold - num_of_servers)
        for states_to_wait in range(threshold, system_capacity + 1)
    ]
)
sojourn_array = sojourn_array * sojourn_time
sojourn_array


array([1.66666667, 1.66666667, 1.66666667, 1.66666667])

In [193]:
# Waiting states probability
pi_array[:-1, min(num_of_servers, threshold) :]


array([[0.13352132, 0.11126777, 0.03814895, 0.01430586],
       [       nan, 0.05457419, 0.02988334, 0.01478272],
       [       nan, 0.03308008, 0.0207064 , 0.01146058],
       [       nan, 0.02174902, 0.01415437, 0.00817303],
       [       nan, 0.01469214, 0.00967368, 0.00567089],
       [       nan, 0.01001224, 0.00661605, 0.00389874],
       [       nan, 0.00684234, 0.00452654, 0.00267214]])

In [194]:
# Waiting states probability * sojourn_array
pi_array[:-1, min(num_of_servers, threshold) :] * sojourn_array


array([[0.22253553, 0.18544628, 0.06358158, 0.02384309],
       [       nan, 0.09095698, 0.04980557, 0.02463786],
       [       nan, 0.05513347, 0.03451067, 0.01910097],
       [       nan, 0.03624837, 0.02359062, 0.01362172],
       [       nan, 0.0244869 , 0.0161228 , 0.00945148],
       [       nan, 0.01668706, 0.01102674, 0.0064979 ],
       [       nan, 0.0114039 , 0.00754423, 0.00445356]])

In [195]:
# Sum of the above matrix
np.nansum(pi_array[:-1, min(num_of_servers, threshold) :] * sojourn_array)


0.9506873106884532

In [196]:
# Probability of accepting state
np.nansum(pi_array[:-1, :])


0.9873193539136187

In [197]:
# Sum of the above matrix
np.nansum(pi_array[:-1, min(num_of_servers, threshold) :] * sojourn_array) / np.nansum(
    pi_array[:-1, :]
)


0.962897472757968

# Combined functions


In [198]:
lambda_2 = 0.2
lambda_1 = 0.3
mu = 0.07
num_of_servers = 7
threshold = 10
system_capacity = 15
buffer_capacity = 10

output = "ambulance"


In [199]:
def get_closed_form_waiting_time(
    lambda_2,
    lambda_1,
    mu,
    num_of_servers,
    threshold,
    system_capacity,
    buffer_capacity,
    output="both",
):

    all_states = abg.markov.build_states(
        threshold=threshold,
        system_capacity=system_capacity,
        buffer_capacity=buffer_capacity,
    )
    transition_matrix = abg.markov.get_transition_matrix(
        lambda_2=lambda_2,
        lambda_1=lambda_1,
        mu=mu,
        num_of_servers=num_of_servers,
        threshold=threshold,
        system_capacity=system_capacity,
        buffer_capacity=buffer_capacity,
    )
    pi = abg.markov.get_steady_state_algebraically(Q=transition_matrix)
    pi_array = abg.markov.get_markov_state_probabilities(pi, all_states)

    sojourn_time = 1 / (num_of_servers * mu)

    if output == "others":
        sojourn_array = (
            np.array(
                [
                    [
                        states_to_wait
                        for states_to_wait in range(
                            1, system_capacity - num_of_servers + 1
                        )
                    ]
                ]
            )
            * sojourn_time
        )  # for states_column in range(buffer_capacity + 1)]) * sojourn_time
        mean_waiting_time = np.nansum(
            pi_array[:, num_of_servers:-1] * sojourn_array
        ) / np.nansum(pi_array[:, :-1])
        return mean_waiting_time

    if output == "ambulance":
        sojourn_array = (
            np.array(
                [
                    states_to_wait
                    for states_to_wait in range(1, threshold - num_of_servers + 1)
                ]
                + [
                    max(0, threshold - num_of_servers)
                    for states_to_wait in range(threshold, system_capacity + 1)
                ]
            )
            * sojourn_time
        )
        if threshold > system_capacity:
            mean_waiting_time = np.nansum(
                pi_array[:, min(num_of_servers, threshold) : -1] * sojourn_array[:-1]
            ) / np.nansum(pi_array[:, :-1])
            return mean_waiting_time

        mean_waiting_time = np.nansum(
            pi_array[:-1, min(num_of_servers, threshold) :] * sojourn_array
        ) / np.nansum(pi_array[:-1, :])
        return mean_waiting_time


###### Others


In [200]:
get_closed_form_waiting_time(
    lambda_2=lambda_2,
    lambda_1=lambda_1,
    mu=mu,
    num_of_servers=num_of_servers,
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
    output="others",
)


8.0742838049914

In [201]:
abg.markov.get_mean_waiting_time_markov(
    lambda_2=lambda_2,
    lambda_1=lambda_1,
    mu=mu,
    num_of_servers=num_of_servers,
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
    output="others",
)


8.074283804991397

##### Ambulance


In [202]:
get_closed_form_waiting_time(
    lambda_2=lambda_2,
    lambda_1=lambda_1,
    mu=mu,
    num_of_servers=num_of_servers,
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
    output="ambulance",
)


4.8691430439364005

In [203]:
abg.markov.get_mean_waiting_time_markov(
    lambda_2=lambda_2,
    lambda_1=lambda_1,
    mu=mu,
    num_of_servers=num_of_servers,
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
    output="ambulance",
)


4.869143043936401

In [205]:
for num_of_servers in range(5, 10):
    for threshold in range(7, 12):
        for system_capacity in range(15, 20):
            for buffer_capacity in range(5, 10):
                closed_other = get_closed_form_waiting_time(
                    lambda_2=lambda_2,
                    lambda_1=lambda_1,
                    mu=mu,
                    num_of_servers=num_of_servers,
                    threshold=threshold,
                    system_capacity=system_capacity,
                    buffer_capacity=buffer_capacity,
                    output="others",
                )
                recurs_other = abg.markov.get_mean_waiting_time_markov(
                    lambda_2=lambda_2,
                    lambda_1=lambda_1,
                    mu=mu,
                    num_of_servers=num_of_servers,
                    threshold=threshold,
                    system_capacity=system_capacity,
                    buffer_capacity=buffer_capacity,
                    output="others",
                )
                if round(closed_other, 6) != round(recurs_other, 6):
                    print(
                        "O",
                        num_of_servers,
                        threshold,
                        system_capacity,
                        buffer_capacity,
                    )

                closed_amb = get_closed_form_waiting_time(
                    lambda_2=lambda_2,
                    lambda_1=lambda_1,
                    mu=mu,
                    num_of_servers=num_of_servers,
                    threshold=threshold,
                    system_capacity=system_capacity,
                    buffer_capacity=buffer_capacity,
                    output="ambulance",
                )
                recurs_amb = abg.markov.get_mean_waiting_time_markov(
                    lambda_2=lambda_2,
                    lambda_1=lambda_1,
                    mu=mu,
                    num_of_servers=num_of_servers,
                    threshold=threshold,
                    system_capacity=system_capacity,
                    buffer_capacity=buffer_capacity,
                    output="ambulance",
                )
                if round(closed_amb, 6) != round(recurs_amb, 6):
                    print(
                        "A",
                        num_of_servers,
                        threshold,
                        system_capacity,
                        buffer_capacity,
                    )
